# Creating a Placement Algorithm

This tutorial demonstrates how we can create a simple placement algorithm on EdgeSimPy.

Let's start by importing the EdgeSimPy modules:

In [75]:
# EdgeSimPy Import Debugging Script

# Explicit dependency installation

!pip install tqdm
!pip install numpy rich
!pip install rich
!pip install rich --upgrade
!pip install networkx==2.6.2
!pip install matplotlib pandas numpy
!pip install git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

# Python and package information
!python --version
!pip list | grep -E "networkx|edge_sim_py"

# Comprehensive import and debugging script
import sys
import os
import importlib

def print_module_structure(module_name):
    """
    Recursively print the structure of a module
    """
    print(f"\n--- Module Structure for {module_name} ---")
    try:
        # Import the module
        module = importlib.import_module(module_name)

        # Get the module's file path
        module_file = getattr(module, '__file__', 'No __file__ attribute')
        print(f"Module file path: {module_file}")

        # Get the module's directory
        module_dir = os.path.dirname(module_file) if hasattr(module, '__file__') else 'Unknown'
        print(f"Module directory: {module_dir}")

        # List all attributes and their types
        print("\nModule Contents:")
        for attr_name in dir(module):
            try:
                attr = getattr(module, attr_name)
                print(f"  {attr_name}: {type(attr)}")
            except Exception as attr_err:
                print(f"  {attr_name}: Could not retrieve (Error: {attr_err})")

        # List files in the module directory
        if os.path.isdir(module_dir):
            print("\nFiles in module directory:")
            try:
                for item in os.listdir(module_dir):
                    print(f"  {item}")
            except Exception as list_err:
                print(f"  Could not list directory contents: {list_err}")

    except ImportError as e:
        print(f"Could not import {module_name}: {e}")
    except Exception as e:
        print(f"Unexpected error examining {module_name}: {e}")

# Print Python path and sys.path for debugging
print("--- Python Path ---")
print(sys.path)

# Attempt to import and examine EdgeSimPy
print_module_structure('edge_sim_py')

# Attempt alternative import methods
print("\n--- Alternative Import Attempts ---")
import_attempts = [
    'edge_sim_py',
    'edge_sim_py.core',
    'edge_sim_py.components',
    'edge_sim_py.device',
    'edge_sim_py.server'
]

for attempt in import_attempts:
    print(f"\nTrying to import {attempt}")
    try:
        module = importlib.import_module(attempt)
        print(f"Successfully imported {attempt}")
        print(f"Module file: {getattr(module, '__file__', 'No file attribute')}")
    except ImportError as e:
        print(f"Import failed: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# List installed packages with their paths
print("\n--- Installed Packages Paths ---")
for package_name in ['edge_sim_py', 'networkx', 'numpy', 'pandas']:
    try:
        package = importlib.import_module(package_name)
        print(f"{package_name}: {package.__file__}")
    except ImportError:
        print(f"{package_name}: Not found")
    except Exception as e:
        print(f"{package_name}: Error - {e}")


  Cloning https://github.com/EdgeSimPy/EdgeSimPy.git (to revision v1.1.0) to /tmp/pip-req-build-28pc6bu_
  Running command git clone --filter=blob:none --quiet https://github.com/EdgeSimPy/EdgeSimPy.git /tmp/pip-req-build-28pc6bu_
  Running command git checkout -q 5ea400b39390490b25dabf8be711fe559cb2cbff
  Resolved https://github.com/EdgeSimPy/EdgeSimPy.git to commit 5ea400b39390490b25dabf8be711fe559cb2cbff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Python 3.11.11
edge_sim_py                        1.1.0
networkx                           2.6.2
--- Python Path ---
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']

-

## Implementing the Placement Algorithm

In this example, we are going to create a simple placement algorithm that works according to the well-known First-Fit heuristic. In a nutshell, our algorithm will provision each service to the first edge server with available resources to host them.

In [8]:
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## Running the Simulation

As we're creating a placement algorithm, we must instruct EdgeSimPy that it needs to continue the simulation until all services are provisioned within the infrastructure.

To do so, let's create a simple function that will be used as the simulation's stopping criterion. EdgeSimPy will run that function at the end of each time step, halting the simulation as soon as it returns `True`.

In [6]:
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0

    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1

    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Google Colab Setup for FCFS Task Processing



In [32]:
# Google Colab Setup for FCFS Task Processing

# Install required libraries
!pip install numpy

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# List files in the task sets directory
import os
task_sets_dir = '/content/drive/My Drive/FCFS_Task_Sets/'
print("Available task set files:")
for filename in os.listdir(task_sets_dir):
    print(filename)

# Note: After running this, copy the full path of the desired JSON file
# and use it in the main FCFS scheduler script

MessageError: Error: credential propagation was unsuccessful

FCFS Algorithm Logic


In [9]:
import json
from typing import List, Dict, Any
import logging
import sys
import time

# Configure logging to print to console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)  # Explicitly add console output
    ]
)
logger = logging.getLogger(__name__)

# Print function to ensure output
def print_to_console(*args, **kwargs):
    """
    Wrapper function to ensure printing
    """
    print(*args, **kwargs)
    sys.stdout.flush()

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')
        self.task_class = self.details.get('task_class', 'generic')
        self.cpu_intensity = self.details.get('cpu_intensity', 'medium')

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with enhanced tracking and visualization
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.memory = memory
        self.bandwidth = bandwidth

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)

    def process_queue(self, current_time: float):
        """
        Process tasks in the queue with detailed tracking
        """
        # Process current tasks first
        for task in self.current_tasks[:]:
            processing_result = task.process(self.cpu_rating)

            # Detailed task processing output
            self._log_task_processing(task, processing_result)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

        # If resource has available capacity, move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:  # Limit concurrent tasks
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)

        return len(self.current_tasks)

    def _log_task_processing(self, task: Task, processing_result: Dict):
        """
        Log detailed task processing information
        """
        print_to_console(
            f"Resource {self.id} ({self.type}) - "
            f"Task {task.id} ({task.task_name}): "
            f"Processed {processing_result['processed']:.2f} MI, "
            f"Remaining {processing_result['remaining']:.2f} MI, "
            f"Completion: {processing_result['completion_percentage']:.2f}%"
        )

class AdvancedFCFSScheduler:
    """
    Advanced First-Come-First-Serve Scheduler with Real-Time Visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.task_queue: List[Task] = []
        self.current_time = 0

        # Metrics tracking with enhanced details
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'queued_tasks': 0,
            'task_distribution': {},
            'resource_utilization': {},
            'average_wait_time': 0,
            'max_wait_time': 0
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks from JSON with comprehensive parsing
        """
        print_to_console(f"Attempting to load tasks from: {json_path}")

        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),  # Default 10 MB
                cpu_required=task_dict.get('instructions', 50000),  # Default 50,000 MI
                task_details=task_dict
            )
            task.arrival_time = self.current_time
            tasks.append(task)

        print_to_console(f"Loaded {len(tasks)} tasks from JSON")
        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources with advanced visualization
        """
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin task distribution with visualization
        resource_index = 0
        for task in tasks:
            # Select resource
            resource = self.resources[resource_index]

            # Enqueue task
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1

            # Cycle through resources
            resource_index = (resource_index + 1) % len(self.resources)

        # Update metrics
        self.metrics['task_distribution'] = task_distribution
        self.metrics['queued_tasks'] = sum(len(resource.task_queue) for resource in self.resources)

        # Print initial distribution
        print_to_console("\n--- Initial Task Distribution ---")
        for resource_type, count in task_distribution.items():
            print_to_console(f"{resource_type}: {count} tasks")

        print_to_console("\n--- Resource Queue Lengths ---")
        for i, resource in enumerate(self.resources, 1):
            print_to_console(f"Resource {i} ({resource.type}) Queue Length: {len(resource.task_queue)} tasks")

    def run_simulation(self, max_iterations: int = 1000):
        """
        Run scheduling simulation with real-time visualization
        """
        # Distribute tasks initially
        self.distribute_tasks()

        # Simulation loop with enhanced visualization
        start_time = time.time()
        for iteration in range(max_iterations):
            print_to_console(f"\n--- Iteration {iteration} ---")

            # Process queues for all resources
            completed_in_iteration = 0
            resource_utilization = {}

            for resource in self.resources:
                # Track resource utilization
                initial_completed = len(resource.completed_tasks)
                resource.process_queue(self.current_time)
                completed_this_resource = len(resource.completed_tasks) - initial_completed
                completed_in_iteration += completed_this_resource

                # Calculate resource utilization
                resource_utilization[resource.type] = {
                    'completed_tasks': completed_this_resource,
                    'current_tasks': len(resource.current_tasks),
                    'queue_length': len(resource.task_queue)
                }

            # Update metrics
            self.metrics['completed_tasks'] = sum(
                len(resource.completed_tasks) for resource in self.resources
            )
            self.metrics['resource_utilization'] = resource_utilization

            # Print real-time resource utilization
            print_to_console("\n--- Resource Utilization ---")
            for resource_type, stats in resource_utilization.items():
                print_to_console(
                    f"{resource_type}: "
                    f"Completed: {stats['completed_tasks']}, "
                    f"Current Tasks: {stats['current_tasks']}, "
                    f"Queue Length: {stats['queue_length']}"
                )

            # Check if all tasks are processed
            if self.metrics['completed_tasks'] == self.metrics['total_tasks']:
                print_to_console("\n--- All Tasks Processed! ---")
                break

            # Increment time
            self.current_time += 1

            # Optional: Add a small delay to simulate real-time processing
            time.sleep(0.1)

        # Calculate total processing time
        total_processing_time = time.time() - start_time
        self.metrics['total_processing_time'] = total_processing_time

        # Calculate wait time metrics
        self.calculate_wait_time_metrics()

        return self.metrics

    def calculate_wait_time_metrics(self):
        """
        Calculate comprehensive wait time metrics
        """
        all_completed_tasks = []
        for resource in self.resources:
            all_completed_tasks.extend(resource.completed_tasks)

        if all_completed_tasks:
            wait_times = [task.wait_time for task in all_completed_tasks]
            self.metrics['average_wait_time'] = sum(wait_times) / len(wait_times)
            self.metrics['max_wait_time'] = max(wait_times)

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Explicitly set print to console
    print = print_to_console

    # Create resources matching original configuration
    resources = create_original_resources()

    # Print initial resource details
    print("\n--- Resource Configurations ---")
    for resource in resources:
        print(f"Resource {resource.id} ({resource.type}):")
        print(f"  CPU Rating: {resource.cpu_rating} MI/s")
        print(f"  Memory: {resource.memory} GB")
        print(f"  Bandwidth: {resource.bandwidth} MB/s")

    # Initialize scheduler
    scheduler = AdvancedFCFSScheduler(resources)

    # Run simulation
    metrics = scheduler.run_simulation()

    # Print final detailed metrics
    print("\n--- Final Scheduling Metrics ---")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    # Detailed resource reporting
    print("\n--- Final Resource Status ---")
    for resource in scheduler.resources:
        print(f"\nResource {resource.id} ({resource.type}):")
        print(f"Completed Tasks: {len(resource.completed_tasks)}")
        print(f"Remaining Queue: {len(resource.task_queue)}")

if __name__ == "__main__":
    main()



--- Resource Configurations ---
Resource 1 (Edge_Raspberry_Pi):
  CPU Rating: 80000 MI/s
  Memory: 1 GB
  Bandwidth: 5 MB/s
Resource 2 (Edge_Smartphone):
  CPU Rating: 400000 MI/s
  Memory: 4 GB
  Bandwidth: 20 MB/s
Resource 3 (Cloud_Host):
  CPU Rating: 1000000 MI/s
  Memory: 32 GB
  Bandwidth: 80 MB/s
Attempting to load tasks from: /content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'

Using Rich in FCFS

In [33]:
import json
import numpy as np
from typing import List, Dict, Any
import logging
import sys
import time
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 task_type: str,
                 data_size: float,     # in GB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.type = task_type
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = f"{task_type}_Task_{task_id}"

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with comprehensive tracking and utilization metrics
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int,     # in MB/s
                 num_cpus: int = 1):  # Number of CPUs, defaulting to 1
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.total_memory = memory
        self.bandwidth = bandwidth

        # CPU configuration
        self.num_cpus = num_cpus
        self.available_cpus = num_cpus

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Utilization tracking
        self.current_cpu_usage = 0
        self.current_memory_usage = 0

        # Additional tracking
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

    def can_process_task(self, task: Task) -> bool:
        """
        Check if the resource can process the given task
        """
        # Check if CPUs are available
        if self.available_cpus <= 0:
            return False

        # Cloud host can process all task types
        if self.type == "Cloud_Host":
            return True

        # Edge nodes (Raspberry Pi and Smartphone) can only process RT2 tasks
        if self.type in ["Edge_Raspberry_Pi", "Edge_Smartphone"]:
            # Explicitly fail RT1 and RT3 tasks on edge resources
            if task.type in ["RT1", "RT3"]:
                logger.warning(f"Task {task.id} of type {task.type} FAILED on {self.type}")
                return False

            # Additional memory check for RT2 tasks
            if task.data_size > self.total_memory:
                logger.warning(f"Task {task.id} requires {task.data_size} GB, exceeding {self.type}'s memory of {self.total_memory} GB")
                return False

        return True

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue if it can be processed
        """
        if self.can_process_task(task):
            task.queue_position = len(self.task_queue)
            task.arrival_time = time.time()
            self.task_queue.append(task)
        else:
            # Mark task as failed
            task.status = 'failed'

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in the queue with detailed tracking and utilization update
        """
        # Reset current usage and task tracking
        self.current_cpu_usage = 0
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

        # Reset available CPUs
        self.available_cpus = self.num_cpus

        # Process current tasks first
        for task in self.current_tasks[:]:
            # Skip if no CPUs available
            if self.available_cpus <= 0:
                break

            # Determine how much CPU can be used for this task
            task_cpu = min(self.cpu_rating, task.remaining_cpu)

            processing_result = task.process(task_cpu)

            # Update CPU usage
            processed_amount = processing_result['processed']
            self.current_cpu_usage += processed_amount

            # Track detailed task information
            task_info = {
                'id': task.id,
                'name': task.task_name,
                'type': task.type,
                'processed': processed_amount,
                'total_required': task.total_cpu_required,
                'completion_percentage': processing_result['completion_percentage']
            }
            self.detailed_task_tracking.append(task_info)

            # Calculate task CPU demand
            task_demand = processed_amount / self.cpu_rating
            self.task_cpu_demands.append(task_demand)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)
                # Free up a CPU
                self.available_cpus += 1

        # Move tasks from queue to current tasks if CPUs are available
        while self.task_queue and self.available_cpus > 0:
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)
            # Use up a CPU
            self.available_cpus -= 1

        # Calculate CPU utilization
        if self.task_cpu_demands:
            cpu_utilization = min(sum(self.task_cpu_demands) * 100, 100)
        else:
            cpu_utilization = 0

        # Estimate memory usage
        self.current_memory_usage = len(self.current_tasks) * (self.total_memory / 10)
        memory_utilization = min((self.current_memory_usage / self.total_memory) * 100, 100)

        # Return detailed resource state with utilization
        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'cpu_utilization': cpu_utilization,
            'memory_utilization': memory_utilization,
            'raw_cpu_usage': self.current_cpu_usage,
            'task_demands': self.task_cpu_demands,
            'detailed_tasks': self.detailed_task_tracking,
            'available_cpus': self.available_cpus
        }

def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Edge Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Edge_{i}",
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=20         # 20 MB/s
            )
        )

    # Create 5 Raspberry Pi Edge Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=5          # 5 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=80         # 80 MB/s
            )
        )

    return resources

class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        self.console = Console()

        # Poisson process parameters
        self.arrival_rate = 0.8  # λ = 0.8 tasks per second

        # Metrics tracking
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'failed_tasks': 0,
            'task_distribution': {},
            'makespan': 0,
            'throughput': 0
        }

    def generate_tasks(self, simulation_time: float = 100) -> List[Task]:
        """
        Generate tasks using Poisson process
        """
        # Task types and their characteristics based on the paper
        task_types = [
            # Read Tasks
            {"type": "RT1", "data_size": 5.0, "cpu_required": 2_000_000},   # CPU-intensive, memory-intensive
            {"type": "RT2", "data_size": 0.2, "cpu_required": 4_000_000},   # CPU-intensive, memory-light
            {"type": "RT3", "data_size": 5.0, "cpu_required": 200_000},     # CPU-light, memory-intensive
            {"type": "RT4", "data_size": 0.5, "cpu_required": 500_000}      # CPU-light, memory-light
        ]

        # Generate task arrival times using Poisson process
        num_tasks = np.random.poisson(self.arrival_rate * simulation_time)

        tasks = []
        for i in range(num_tasks):
            # Randomly select task type
            task_type_data = np.random.choice(task_types)

            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )

            # Set arrival time
            task.arrival_time = np.random.uniform(0, simulation_time)

            tasks.append(task)

        # Sort tasks by arrival time
        return sorted(tasks, key=lambda x: x.arrival_time)

    def distribute_tasks(self, total_tasks: int = 1500) -> List[Task]:
        """
        Generate and distribute a fixed number of tasks
        """
        # Task types and their characteristics based on the paper
        task_types = [
            # Read Tasks
            {"type": "RT1", "data_size": 5.0, "cpu_required": 2_000_000},   # CPU-intensive, memory-intensive
            {"type": "RT2", "data_size": 0.2, "cpu_required": 4_000_000},   # CPU-intensive, memory-light
            {"type": "RT3", "data_size": 5.0, "cpu_required": 200_000},     # CPU-light, memory-intensive
            {"type": "RT4", "data_size": 0.5, "cpu_required": 500_000}      # CPU-light, memory-light
        ]

        # Generate tasks
        tasks = []
        for i in range(total_tasks):
            # Randomly select task type
            task_type_data = np.random.choice(task_types)

            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )

            # Set arrival time with uniform distribution
            task.arrival_time = np.random.uniform(0, 100)  # Distribute over 100 seconds

            tasks.append(task)

        # Sort tasks by arrival time
        tasks.sort(key=lambda x: x.arrival_time)

        # Update metrics
        self.metrics['total_tasks'] = total_tasks

        # Explicitly select edge, raspberry, and cloud resources
        edge_resources = [r for r in self.resources if r.type.startswith("Edge_")]
        raspberry_resources = [r for r in self.resources if r.type.startswith("Raspberry_")]
        cloud_resources = [r for r in self.resources if r.type.startswith("Cloud_")]

        # Define resource order as specified in the paper
        # Order: Smartphone, Raspberry Pi, Cloud
        resource_order = edge_resources + raspberry_resources + cloud_resources

        # Track task distribution and failures
        task_distribution = {resource.type: 0 for resource in self.resources}
        failed_tasks_by_resource = {resource.type: 0 for resource in self.resources}

        # Circular resource selection
        resource_index = 0
        num_resources = len(resource_order)

        for task in tasks:
            # Select resource in the specified order
            resource = resource_order[resource_index]

            # Attempt to enqueue task
            if resource.can_process_task(task):
                resource.enqueue_task(task)
                task_distribution[resource.type] += 1
            else:
                # Increment failed tasks for the specific resource type
                failed_tasks_by_resource[resource.type] += 1
                self.metrics['failed_tasks'] += 1

            # Move to next resource in circular manner
            resource_index = (resource_index + 1) % num_resources

        self.metrics['task_distribution'] = task_distribution

        # Print distribution table
        distribution_table = Table(title="Task Distribution")
        distribution_table.add_column("Resource", style="cyan")
        distribution_table.add_column("Tasks", style="magenta")
        distribution_table.add_column("Failed Tasks", style="red")

        for resource_type in task_distribution:
            distribution_table.add_row(
                resource_type,
                str(task_distribution[resource_type]),
                str(failed_tasks_by_resource[resource_type])
            )

        self.console.print(distribution_table)

        return tasks
    def run_simulation(self, total_tasks: int = 1500, max_iterations: int = 10000):
            """
            Run simulation with a fixed number of tasks
            """
            # Record start time
            self.start_time = time.time()

            # Distribute tasks
            tasks = self.distribute_tasks(total_tasks)

            # Prepare layout for live visualization
            layout = Layout()
            layout.split_row(
                Layout(name="resource1"),
                Layout(name="resource2"),
                Layout(name="resource3")
            )

            # Live visualization
            with Live(layout, console=self.console, refresh_per_second=10) as live:
                for iteration in range(max_iterations):
                    # Calculate current simulation time
                    current_simulation_time = time.time() - self.start_time

                    # Process tasks on each resource
                    for i, resource in enumerate(self.resources, 1):
                        # Process resource queue
                        resource_status = resource.process_queue(current_simulation_time)

                        # Update layout with resource-specific panel
                        layout[f"resource{i}"].update(
                            self._create_resource_panel(resource, resource_status)
                        )

                    # Update live display
                    live.update(layout)

                    # Track completed and failed tasks across all resources
                    total_processed_tasks = sum(
                        len(resource.completed_tasks) for resource in self.resources
                    ) + self.metrics['failed_tasks']

                    # Check if all tasks are processed (completed or failed)
                    if total_processed_tasks >= total_tasks:
                        logger.info(f"Simulation completed in {iteration} iterations")
                        break

                    time.sleep(0.1)

            # Calculate final metrics
            completed_tasks = sum(len(resource.completed_tasks) for resource in self.resources)
            self.metrics['completed_tasks'] = completed_tasks
            self.metrics['makespan'] = time.time() - self.start_time

            return self.metrics
    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with utilization metrics
        """
        # Create table for resource details
        table = Table(show_header=False)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['cpu_utilization']:.2f}% "
            f"({status['raw_cpu_usage']:.2f}/{resource.cpu_rating} MI/s)"
        )

        # Detailed task tracking
        if status['detailed_tasks']:
            table.add_row("\n[bold]Current Tasks[/bold]")
            for task in status['detailed_tasks']:
                table.add_row(                f"[blue]Task {task['id']} ({task['type']}):[/blue] "
                f"Processed {task['processed']:.2f}/{task['total_required']} MI "
                f"({task['completion_percentage']:.2f}%)"
                )

        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['memory_utilization']:.2f}% "
            f"({resource.current_memory_usage:.2f}/{resource.total_memory} GB)"
        )

        # Task processing status
        table.add_row("\n[bold]Task Processing[/bold]")
        table.add_row(f"[green]Completed Tasks:[/green] {status['completed_tasks']}")
        table.add_row(f"[yellow]Current Tasks:[/yellow] {status['current_tasks']}")
        table.add_row(f"[red]Queue Length:[/red] {status['queue_length']}")

        # Create panel with resource-specific styling
        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )

def create_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    """
    Main simulation entry point
    """
    # Create resources
    resources = create_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    # Run simulation
    try:
        # Run simulation for 1500 tasks
        metrics = scheduler.run_simulation(total_tasks=1500)

        # Print final metrics
        print("\n--- Simulation Metrics ---")
        print(f"Total Tasks Generated: {metrics['total_tasks']}")
        print(f"Completed Tasks: {metrics['completed_tasks']}")
        print(f"Failed Tasks: {metrics['failed_tasks']}")
        print("\nTask Distribution:")
        for resource_type, count in metrics['task_distribution'].items():
            print(f"{resource_type}: {count}")

        print("\nPerformance Metrics:")
        print(f"Makespan: {metrics['makespan']:.2f} seconds")

    except Exception as e:
        logger.error(f"Simulation failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


             Task Distribution              
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Resource          ┃ Tasks ┃ Failed Tasks ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ Edge_Raspberry_Pi │ 231   │ 269          │
│ Edge_Smartphone   │ 251   │ 249          │
│ Cloud_Host        │ 500   │ 0            │
└───────────────────┴───────┴──────────────┘

Output()


--- Simulation Metrics ---
Total Tasks Generated: 1500
Completed Tasks: 982
Failed Tasks: 518

Task Distribution:
Edge_Raspberry_Pi: 231
Edge_Smartphone: 251
Cloud_Host: 500

Performance Metrics:
Makespan: 17.95 seconds


With CPU and Utilization output included

In [89]:
import json
from typing import List, Dict, Any
import logging
import sys
import time
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
from rich.progress import Progress, SpinnerColumn, TextColumn, BarColumn, TaskProgressColumn

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')
        self.task_class = self.details.get('task_class', 'generic')
        self.cpu_intensity = self.details.get('cpu_intensity', 'medium')

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with comprehensive tracking and utilization metrics
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.total_memory = memory
        self.bandwidth = bandwidth

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Utilization tracking
        self.current_cpu_usage = 0
        self.current_memory_usage = 0

        # Additional tracking for more nuanced CPU utilization
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in the queue with detailed tracking and utilization update
        """
        # Reset current usage and task tracking
        self.current_cpu_usage = 0
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

        # Calculate available CPU for this time step
        available_cpu = self.cpu_rating

        # Process current tasks first
        for task in self.current_tasks[:]:
            # Determine how much CPU can be used for this task
            task_cpu = min(available_cpu, task.remaining_cpu)

            processing_result = task.process(task_cpu)

            # Update CPU usage and available CPU
            processed_amount = processing_result['processed']
            self.current_cpu_usage += processed_amount
            available_cpu -= processed_amount

            # Track detailed task information
            task_info = {
                'id': task.id,
                'name': task.task_name,
                'processed': processed_amount,
                'total_required': task.total_cpu_required,
                'completion_percentage': processing_result['completion_percentage']
            }
            self.detailed_task_tracking.append(task_info)

            # Calculate task CPU demand
            task_demand = processed_amount / self.cpu_rating
            self.task_cpu_demands.append(task_demand)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

            # Stop processing if no CPU left
            if available_cpu <= 0:
                break

        # Calculate CPU utilization
        # Use sum of task CPU demands to get a more dynamic representation
        if self.task_cpu_demands:
            cpu_utilization = min(sum(self.task_cpu_demands) * 100, 100)
        else:
            cpu_utilization = 0

        # Estimate memory usage (simple model: each current task uses some memory)
        self.current_memory_usage = len(self.current_tasks) * (self.total_memory / 10)
        memory_utilization = min((self.current_memory_usage / self.total_memory) * 100, 100)

        # If resource has available capacity, move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:  # Limit concurrent tasks
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)

        # Return detailed resource state with utilization
        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'cpu_utilization': cpu_utilization,
            'memory_utilization': memory_utilization,
            'raw_cpu_usage': self.current_cpu_usage,
            'task_demands': self.task_cpu_demands,
            'detailed_tasks': self.detailed_task_tracking
        }

class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        self.console = Console()

        # Metrics tracking
        self.metrics = {
            'total_tasks': 0,
            'task_distribution': {},
            'resource_status': {}
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks from JSON
        """
        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),
                cpu_required=task_dict.get('instructions', 50000),
                task_details=task_dict
            )
            task.arrival_time = self.current_time
            tasks.append(task)

        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources
        """
        # Load tasks
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin distribution
        resource_index = 0
        for task in tasks:
            resource = self.resources[resource_index]
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1
            resource_index = (resource_index + 1) % len(self.resources)

        self.metrics['task_distribution'] = task_distribution

        # Print distribution table
        distribution_table = Table(title="Task Distribution")
        distribution_table.add_column("Resource", style="cyan")
        distribution_table.add_column("Tasks", style="magenta")

        for resource_type, count in task_distribution.items():
            distribution_table.add_row(resource_type, str(count))

        self.console.print(distribution_table)

    def run_simulation(self, max_iterations: int = 10000):
        """
        Run simulation with a stopping criterion similar to the provided code
        """
        # Distribute tasks
        self.distribute_tasks()

        # Total number of tasks
        total_tasks = self.metrics['total_tasks']

        # Prepare layout for live visualization
        layout = Layout()
        layout.split_row(
            Layout(name="resource1"),
            Layout(name="resource2"),
            Layout(name="resource3")
        )

        # Live visualization
        with Live(layout, console=self.console, refresh_per_second=10) as live:
            for iteration in range(max_iterations):
                self.current_time += 1

                # Process tasks on each resource
                for i, resource in enumerate(self.resources, 1):
                    # Process resource queue
                    resource_status = resource.process_queue(self.current_time)

                    # Update layout with resource-specific panel
                    layout[f"resource{i}"].update(
                        self._create_resource_panel(resource, resource_status)
                    )

                # Update live display
                live.update(layout)

                # Custom stopping criterion similar to the provided code
                provisioned_tasks = sum(
                    len(resource.completed_tasks) for resource in self.resources
                )

                # Stop when all tasks are provisioned (completed)
                if provisioned_tasks == total_tasks:
                    logger.info(f"Simulation completed in {iteration} iterations")
                    break

                time.sleep(0.1)

        return self.metrics

    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with utilization metrics
        """
        # Create table for resource details
        table = Table(show_header=False)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['cpu_utilization']:.2f}% "
            f"({status['raw_cpu_usage']:.2f}/{resource.cpu_rating} MI/s)"
        )

        # Show individual task demands for more insight
        if status['task_demands']:
            demands_str = ", ".join([f"{d*100:.2f}%" for d in status['task_demands']])
            table.add_row(f"[yellow]Task Demands:[/yellow] {demands_str}")

        # Detailed task tracking
        if status['detailed_tasks']:
            table.add_row("\n[bold]Current Tasks[/bold]")
            for task in status['detailed_tasks']:
                table.add_row(
                    f"[blue]Task {task['id']} ({task['name']}):[/blue] "
                    f"{task['processed']:.2f}/{task['total_required']} MI "
                    f"({task['completion_percentage']:.2f}%)"
                )

        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['memory_utilization']:.2f}% "
            f"({resource.current_memory_usage:.2f}/{resource.total_memory} GB)"
        )

        # Task processing status
        table.add_row("\n[bold]Task Processing[/bold]")
        table.add_row(f"[green]Completed Tasks:[/green] {status['completed_tasks']}")
        table.add_row(f"[yellow]Current Tasks:[/yellow] {status['current_tasks']}")
        table.add_row(f"[red]Queue Length:[/red] {status['queue_length']}")

        # Create panel with resource-specific styling
        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Create resources
    resources = create_original_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    # Run simulation
    metrics = scheduler.run_simulation()

if __name__ == "__main__":
    main()


      Task Distribution      
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Resource          ┃ Tasks ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Edge_Raspberry_Pi │ 500   │
│ Edge_Smartphone   │ 500   │
│ Cloud_Host        │ 500   │
└───────────────────┴───────┘

Output()

KeyboardInterrupt: 

Using Round Robin Algorithm


In [35]:
import json
import numpy as np
from typing import List, Dict, Any
import logging
import sys
import time
import random
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
import logging
import time
import threading
from google.colab import drive
import os

drive.mount('/content/drive')

# Define log directory in Google Drive
log_dir = "/content/drive/My Drive/EdgeSimPy/logs"

# Create the directory if it does not exist
os.makedirs(log_dir, exist_ok=True)

# List log files in the EdgeSimPy logs directory
print("\n📂 Available log files in EdgeSimPy/logs:")
for filename in os.listdir(log_dir):
    print(filename)
# Configure logging to write to both console and file (in Google Drive)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),  # Output to console
        logging.FileHandler(log_dir)  # Save logs to Google Drive
    ]
)

logger = logging.getLogger(__name__)

class Task:
    def __init__(self,
                 task_id: int,
                 task_type: str,
                 data_size: float,     # in GB
                 cpu_required: float,  # in seconds
                 processing_speed: float = 50000,
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.type = task_type
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = f"{task_type}_Task_{task_id}"
        # Processing control
        self.processing_speed = processing_speed  # Rate of processing per cycle

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        time.sleep(0.1)  # Introduce artificial delay to slow processing
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Log task processing details
        logger.info(f"Processing task {self.id} on resource. Remaining CPU: {self.remaining_cpu}")

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }
class Resource:
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,
                 memory: int,
                 bandwidth: int):
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.total_memory = memory
        self.bandwidth = bandwidth
        self.failed_tasks = 0

        # Task management
        self.task_queue: List[Task] = []
        self.current_task: Optional[Task] = None
        self.completed_tasks: List[Task] = []

        # Utilization tracking
        self.current_cpu_usage = 0
        self.current_memory_usage = 0
    def assign_task(self, task: Task):
        self.task_queue.append(task)
    def can_process_task(self, task: Task) -> bool:
        """
        Check if the resource can process the given task.
        RT1 and RT3 tasks should fail on Edge and Raspberry nodes.
        """
        # Cloud hosts can process all tasks
        if self.type.startswith("Cloud_"):
            return True

        # Edge and Raspberry nodes cannot process RT1 and RT3 tasks
        if self.type.startswith("Edge_") or self.type.startswith("Raspberry_"):
            if task.type in ["RT1", "RT3"]:
                logger.warning(f"Task {task.id} ({task.type}) cannot be processed on {self.type}")
                return False

        # Edge and Raspberry nodes can process RT2, RT4, WT1, WT2, WT3, WT4
        return task.type in ["RT2", "RT4", "WT1", "WT2", "WT3", "WT4"]
    def enqueue_task(self, task: Task):
        """
        Add task to the resource's queue if it can be processed.
        Otherwise, mark the task as failed and increment the failed_tasks counter.
        """
        if self.can_process_task(task):
            self.task_queue.append(task)
            # Log queue length after enqueuing
            logger.info(f"Task {task.id} added to {self.type}. Queue length: {len(self.task_queue)}")
        else:
            task.status = 'failed'  # Mark the task as failed
            self.failed_tasks += 1  # Increment the failed_tasks counter
            logger.warning(f"Task {task.id} ({task.type}) failed on {self.type}")
    def process_queue(self, current_time: float) -> Dict:
        if self.current_task:
            # Process the current task
            task_cpu = min(self.cpu_rating * 0.01, self.current_task.remaining_cpu)
            processing_result = self.current_task.process(task_cpu)
            self.current_cpu_usage = task_cpu
            self.current_memory_usage = self.current_task.data_size

            # Log task processing details
            logger.info(f"Processing task {self.current_task.id} on {self.type}. Remaining CPU: {self.current_task.remaining_cpu}")

            if processing_result['status'] == 'completed':
                self.completed_tasks.append(self.current_task)
                self.current_task = None

        if not self.current_task and self.task_queue:
            # Start a new task if there's no current task
            self.current_task = self.task_queue.pop(0)
            self.current_task.start_time = current_time

            # Log queue length after dequeuing
            logger.info(f"Queue length for {self.type}: {len(self.task_queue)}")

        return {
            'completed_tasks': len(self.completed_tasks),
            'current_task': self.current_task.id if self.current_task else None,
            'queue_length': len(self.task_queue),
            'cpu_utilization': (self.current_cpu_usage / self.cpu_rating) * 100 if self.cpu_rating > 0 else 0,
            'memory_utilization': (self.current_memory_usage / self.total_memory) * 100 if self.total_memory > 0 else 0,
        }
def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Edge Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Edge_{i}",
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=20         # 20 MB/s
            )
        )

    # Create 5 Raspberry Pi Edge Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=5          # 5 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=80         # 80 MB/s
            )
        )

    return resources

class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
            self.resources = resources
            self.current_time = 0
            self.console = Console()
            self.metrics = {
                'total_tasks': 0,
                'completed_tasks': 0,
                'failed_tasks': 0,
                'makespan': 0,
                'throughput': 0
            }

    def generate_tasks(self, simulation_time: float = 100) -> List[Task]:
        """
        Generate tasks using Poisson process
        """
        # Task types and their characteristics based on the paper
        task_types = [
            # Read Tasks
            {"type": "RT1", "data_size": 5.0, "cpu_required": 2_000_000},   # CPU-intensive, memory-intensive
            {"type": "RT2", "data_size": 0.2, "cpu_required": 4_000_000},   # CPU-intensive, memory-light
            {"type": "RT3", "data_size": 5.0, "cpu_required": 200_000},     # CPU-light, memory-intensive
            {"type": "RT4", "data_size": 0.5, "cpu_required": 500_000}      # CPU-light, memory-light
        ]

        # Generate task arrival times using Poisson process
        num_tasks = np.random.poisson(self.arrival_rate * simulation_time)

        tasks = []
        for i in range(num_tasks):
            # Randomly select task type
            task_type_data = np.random.choice(task_types)

            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )

            # Set arrival time
            task.arrival_time = np.random.uniform(0, simulation_time)

            tasks.append(task)

        # Sort tasks by arrival time
        return sorted(tasks, key=lambda x: x.arrival_time)
    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with comprehensive task information
        """
        # Create table for resource details
        table = Table(show_header=False, show_lines=True)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['cpu_utilization']:.2f}% "
            f"({status['raw_cpu_usage']:.2f}/{resource.cpu_rating} MI/s)"
        )

        # Task Queue Information
        table.add_row("\n[bold]Task Queue[/bold]")
        table.add_row(f"[yellow]Queued Tasks:[/yellow] {status['queue_length']}")

        # Current Tasks
        table.add_row("\n[bold]Current Tasks[/bold]")
        if status['detailed_tasks']:
            for task in status['detailed_tasks']:
                # Color-code task stages
                stage_color = {
                    'Started': 'yellow',
                    'Processing': 'blue',
                    'Completed': 'green'
                }
                stage_color_default = 'white'

                stage_color_code = stage_color.get(task.get('current_stage', ''), stage_color_default)

                table.add_row(
                    f"[{stage_color_code}]Task {task['id']} ({task['type']}):[/{stage_color_code}] "
                    f"Stage: {task.get('current_stage', 'Unknown')} "
                    f"Processed {task['processed']:.2f}/{task['total_required']} MI "
                    f"({task['completion_percentage']:.2f}%)"
                )
        else:
            table.add_row("[dim]No tasks currently processing[/dim]")

        # Memory Usage
        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['memory_utilization']:.2f}% "
            f"({resource.current_memory_usage:.2f}/{resource.total_memory} GB)"
        )

        # Task Processing Status
        table.add_row("\n[bold]Task Processing Summary[/bold]")
        table.add_row(f"[green]Completed Tasks:[/green] {status['completed_tasks']}")
        table.add_row(f"[yellow]Current Tasks:[/yellow] {status['current_tasks']}")

        # Detailed task queue information
        table.add_row("\n[bold]Detailed Task Queue[/bold]")
        if resource.task_queue:
            for idx, task in enumerate(resource.task_queue[:5], 1):
                table.add_row(
                    f"[yellow]Queued Task {idx}:[/yellow] "
                    f"ID: {task.id}, Type: {task.type}, "
                    f"CPU Required: {task.total_cpu_required} MI"
                )
            if len(resource.task_queue) > 5:
                table.add_row(f"[dim]... and {len(resource.task_queue) - 5} more tasks[/dim]")
        else:
            table.add_row("[dim]No tasks in queue[/dim]")

        # Create panel with resource-specific styling
        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )
    def distribute_tasks(self, total_tasks: int = 1500) -> List[Task]:
        """
        Generate and distribute tasks to resources using round-robin.
        RT1 and RT3 tasks can be assigned to any resource but will fail on Edge and Raspberry nodes.
        """
        task_types = [
            {"type": "RT1", "data_size": 5.0, "cpu_required": 64.50},
            {"type": "RT2", "data_size": 0.2, "cpu_required": 90.00},
            {"type": "RT3", "data_size": 5.0, "cpu_required": 62.70},
            {"type": "RT4", "data_size": 0.5, "cpu_required": 106.25},
            {"type": "WT1", "data_size": 10.0, "cpu_required": 425.00},
            {"type": "WT2", "data_size": 1.0, "cpu_required": 112.50},
            {"type": "WT3", "data_size": 20.0, "cpu_required": 1006.25},
            {"type": "WT4", "data_size": 0.1, "cpu_required": 42.50}
        ]

        tasks = []
        for i in range(total_tasks):
            task_type_data = random.choice(task_types)
            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )
            task.arrival_time = random.uniform(0, 100)
            tasks.append(task)

        tasks.sort(key=lambda x: x.arrival_time)
        self.metrics['total_tasks'] = total_tasks
        self.metrics['failed_tasks'] = 0  # Reset failed tasks counter

        resource_index = 0  # Single index for round-robin assignment

        for task in tasks:
            assigned = False

            # Try to assign the task to the next resource in round-robin order
            for _ in range(len(self.resources)):
                resource = self.resources[resource_index]
                if resource.can_process_task(task):
                    resource.enqueue_task(task)
                    assigned = True
                    break
                resource_index = (resource_index + 1) % len(self.resources)

            if not assigned:
                self.metrics['failed_tasks'] += 1  # Increment failed tasks counter
                task.status = 'failed'  # Mark the task as failed
                logger.warning(f"Task {task.id} ({task.type}) could not be assigned to any resource")

            # Move to the next resource for the next task
            resource_index = (resource_index + 1) % len(self.resources)

        return tasks
    def run_simulation(self, total_tasks: int = 1500, max_runtime: int = 600):
        tasks = self.distribute_tasks(total_tasks)
        start_time = time.time()

        def generate_live_feed():
            table = Table(title="Live Resource Processing Status")
            table.add_column("Resource", style="cyan")
            table.add_column("Type", style="magenta")
            table.add_column("Current Task", style="yellow")
            table.add_column("Progress", style="green")
            table.add_column("Queue", style="blue")
            table.add_column("Completed", style="cyan")
            table.add_column("Failed", style="red")

            for resource in self.resources:
                status = resource.process_queue(time.time() - start_time)

                # Log queue length and current task details
                logger.info(f"Queue length for {resource.type}: {status['queue_length']}")
                if resource.current_task:
                    logger.info(f"Processing task {resource.current_task.id} on {resource.type}. Remaining CPU: {resource.current_task.remaining_cpu}")

                current_task = "None"
                progress = "N/A"
                if resource.current_task:
                    task = resource.current_task
                    current_task = f"{task.id} ({task.type})"
                    progress = f"{(task.total_cpu_required - task.remaining_cpu) / task.total_cpu_required * 100:.2f}%"

                table.add_row(
                    str(resource.id),
                    resource.type,
                    current_task,
                    progress,
                    str(status['queue_length']),
                    str(len(resource.completed_tasks)),
                    str(resource.failed_tasks)
                )

            return table

        with Live(generate_live_feed(), refresh_per_second=20) as live:
            while time.time() - start_time < max_runtime:
                for resource in self.resources:
                    resource.process_queue(time.time() - start_time)
                live.update(generate_live_feed())

                # Check if all queues are empty and no tasks are being processed
                if all(len(r.task_queue) == 0 and r.current_task is None for r in self.resources):
                    break

                time.sleep(0.1)

        # Calculate final metrics
        self.metrics['completed_tasks'] = sum(len(r.completed_tasks) for r in self.resources)
        self.metrics['failed_tasks'] = sum(r.failed_tasks for r in self.resources)
        self.metrics['makespan'] = time.time() - start_time
        self.metrics['throughput'] = self.metrics['completed_tasks'] / self.metrics['makespan'] if self.metrics['makespan'] > 0 else 0
        return self.metrics
def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Edge Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Edge_{i}",
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=20         # 20 MB/s
            )
        )

    # Create 5 Raspberry Pi Edge Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=5          # 5 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=80         # 80 MB/s
            )
        )

    # Log created resources
    logger.info(f"Created {len(resources)} resources: {[r.type for r in resources]}")

    return resources
def main():
    """
    Main simulation entry point
    """
    # Create resources
    resources = create_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    # Run simulation with 1500 tasks and longer runtime
    try:
        metrics = scheduler.run_simulation(total_tasks=1500, max_runtime=600)

        # Print final metrics
        print("\nFinal Simulation Metrics:")
        for key, value in metrics.items():
            print(f"{key}: {value}")

    except Exception as e:
        logger.error(f"Unhandled error in main: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

MessageError: Error: credential propagation was unsuccessful

Files Categories

In [6]:
import json
from typing import List, Dict, Any
import time
from collections import defaultdict
from rich.console import Console
from rich.table import Table

# Task Specifications
TASK_SPECS = {
    # Read Tasks
    'RT1': {
        'instructions': 2_000_000,
        'data_size': 5,
        'description': 'CPU-intensive, memory-intensive',
        'example': 'Financial modeling based on large historical dataset'
    },
    'RT2': {
        'instructions': 4_000_000,
        'data_size': 0.2,
        'description': 'CPU-intensive, memory-light',
        'example': 'Computation of NP-hard optimization problem'
    },
    'RT3': {
        'instructions': 200_000,
        'data_size': 5,
        'description': 'CPU-light, memory-intensive',
        'example': 'Light database queries on large in-memory dataset'
    },
    'RT4': {
        'instructions': 500_000,
        'data_size': 0.5,
        'description': 'CPU-light, memory-light',
        'example': 'Light video editing'
    },
    # Write Tasks
    'WT1': {
        'instructions': 2_000_000,
        'data_size': 2,
        'description': 'CPU-intensive, I/O-intensive',
        'example': 'Complex data write operations'
    },
    'WT2': {
        'instructions': 1_000_000,
        'data_size': 0.5,
        'description': 'CPU-intensive, I/O-light',
        'example': 'Streamlined data writing'
    },
    'WT3': {
        'instructions': 500_000,
        'data_size': 5,
        'description': 'CPU-light, I/O-intensive',
        'example': 'Bulk data transfer'
    },
    'WT4': {
        'instructions': 200_000,
        'data_size': 0.2,
        'description': 'CPU-light, I/O-light',
        'example': 'Simple data logging'
    }
}

class Task:
    """
    Enhanced Task class with precise categorization
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Detailed metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')

        # Precise task category identification
        self.task_category = self._identify_precise_category()

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

    def _identify_precise_category(self) -> str:
        """
        Identify precise task category based on specifications
        """
        for category, spec in TASK_SPECS.items():
            if (abs(self.total_cpu_required - spec['instructions']) < 1000 and
                abs(self.data_size - spec['data_size']) < 0.1):
                return category
        return 'Uncategorized'

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with advanced task tracking
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.memory = memory
        self.bandwidth = bandwidth

        # Advanced task tracking
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Detailed task categorization tracking
        self.task_category_counts = defaultdict(int)
        self.queue_category_counts = defaultdict(int)
        self.task_details = defaultdict(lambda: {
            'total_instr': 0,
            'total_data_size': 0.0,
            'description': '',
            'example': ''
        })

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue with detailed categorization
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)
        self.queue_category_counts[task.task_category] += 1

        # Track task details
        category = task.task_category
        if category in TASK_SPECS:
            spec = TASK_SPECS[category]
            self.task_details[category]['total_instr'] += task.total_cpu_required
            self.task_details[category]['total_data_size'] += task.data_size
            self.task_details[category]['description'] = spec['description']
            self.task_details[category]['example'] = spec['example']

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks with detailed categorization
        """
        # Process current tasks
        for task in self.current_tasks[:]:
            processing_result = task.process(self.cpu_rating)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

                # Track completed task category
                self.task_category_counts[task.task_category] += 1

        # Move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time

            # Decrement queue category count
            self.queue_category_counts[next_task.task_category] -= 1

            self.current_tasks.append(next_task)

        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'completed_categories': dict(self.task_category_counts),
            'queue_categories': {k: v for k, v in self.queue_category_counts.items() if v > 0}
        }

class DetailedTaskScheduler:
    """
    Scheduler with comprehensive task categorization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.console = Console()
        self.metrics = {
            'total_tasks': 0,
            'task_distribution': {},
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks with comprehensive parsing
        """
        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),
                cpu_required=task_dict.get('instructions', 50000),
                task_details=task_dict
            )
            tasks.append(task)

        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources with categorization
        """
        # Load tasks
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin distribution
        resource_index = 0
        for task in tasks:
            resource = self.resources[resource_index]
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1
            resource_index = (resource_index + 1) % len(self.resources)

        self.metrics['task_distribution'] = task_distribution

        return tasks

    def run_simulation(self, max_iterations: int = 1000):
        """
        Run simulation and track task categorization
        """
        # Distribute tasks
        self.distribute_tasks()

        # Process tasks
        for _ in range(max_iterations):
            all_completed = True

            for resource in self.resources:
                resource.process_queue(time.time())

                # Check if resource still has tasks
                if (len(resource.task_queue) > 0 or
                    len(resource.current_tasks) > 0):
                    all_completed = False

            if all_completed:
                break

        # Generate comprehensive report
        self.generate_final_report()

    def generate_final_report(self):
        """
        Generate detailed report of task processing with comprehensive information
        """
        self.console.rule("[bold blue]Task Processing Report[/bold blue]")

        for resource in self.resources:
            # Create table for resource
            resource_table = Table(title=f"Resource {resource.id}: {resource.type}")
            resource_table.add_column("Task Category", style="cyan")
            resource_table.add_column("Completed Tasks", style="green")
            resource_table.add_column("Remaining in Queue", style="red")
            resource_table.add_column("Total Instructions (MI)", style="magenta")
            resource_table.add_column("Total Data Size (GB)", style="yellow")
            resource_table.add_column("Description", style="blue")

            # Combine all task categories
            all_categories = sorted(set(list(resource.task_category_counts.keys()) +
                                 list(resource.queue_category_counts.keys())))

            # Populate table
            for category in all_categories:
                completed = resource.task_category_counts.get(category, 0)
                queued = resource.queue_category_counts.get(category, 0)

                # Get task details
                details = resource.task_details.get(category, {
                    'total_instr': 0,
                    'total_data_size': 0.0,
                    'description': 'N/A',
                    'example': ''
                })

                resource_table.add_row(
                    category,
                    str(completed),
                    str(queued),
                    f"{details['total_instr']:,}",
                    f"{details['total_data_size']:.2f}",
                    details['description']
                )

            # Print resource-specific table
            self.console.print(resource_table)
            self.console.print("\n")

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Create resources
    resources = create_original_resources()

    # Initialize scheduler
    scheduler = DetailedTaskScheduler(resources)

    # Run simulation
    scheduler.run_simulation()

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'

Task Generator mixed large and small files


In [11]:
import random
import json
from typing import List, Dict, Any
import os
from collections import defaultdict

class TaskGenerator:
    def __init__(self):
        # Detailed task configurations with more specific characteristics
        self.task_configs = {
            # Large Read Tasks
            'large_read_tasks': {
                'RT1': {
                    'instr': 2_000_000,  # Million Instructions
                    'data': 5,           # GB
                    'cpu_intensity': 'high',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-intensive, memory-intensive'
                },
                'RT2': {
                    'instr': 4_000_000,
                    'data': 0.2,
                    'cpu_intensity': 'high',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-intensive, memory-light'
                }
            },
            # Large Write Tasks
            'large_write_tasks': {
                'WT1': {
                    'instr': 2_000_000,
                    'data': 2,
                    'cpu_intensity': 'high',
                    'io_intensity': 'high',
                    'task_class': 'CPU-intensive, I/O-intensive'
                },
                'WT2': {
                    'instr': 1_000_000,
                    'data': 0.5,
                    'cpu_intensity': 'high',
                    'io_intensity': 'low',
                    'task_class': 'CPU-intensive, I/O-light'
                }
            },
            # Small Read Tasks
            'small_read_tasks': {
                'RT3': {
                    'instr': 200_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-light, memory-intensive'
                },
                'RT4': {
                    'instr': 500_000,
                    'data': 0.5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-light, memory-light'
                }
            },
            # Small Write Tasks
            'small_write_tasks': {
                'WT3': {
                    'instr': 500_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'io_intensity': 'high',
                    'task_class': 'CPU-light, I/O-intensive'
                },
                'WT4': {
                    'instr': 200_000,
                    'data': 0.2,
                    'cpu_intensity': 'low',
                    'io_intensity': 'low',
                    'task_class': 'CPU-light, I/O-light'
                }
            }
        }

    def generate_large_task_set(self, total_tasks: int, large_task_percentage: float = 0.7) -> Dict[str, Any]:
        """
        Generate a comprehensive set of tasks with detailed categorization
        """
        # Calculate number of each type
        num_large_tasks = int(total_tasks * large_task_percentage)
        num_small_tasks = total_tasks - num_large_tasks

        # Generate tasks
        tasks = []
        task_id = 1

        # Generate large tasks
        large_tasks = self._generate_tasks(num_large_tasks, "large", task_id)
        tasks.extend(large_tasks)
        task_id += num_large_tasks

        # Generate small tasks
        small_tasks = self._generate_tasks(num_small_tasks, "small", task_id)
        tasks.extend(small_tasks)

        # Shuffle tasks to randomize their order
        random.shuffle(tasks)

        # Categorize tasks
        categorized_tasks = self._categorize_tasks(tasks)

        # Prepare task set metadata
        task_set_metadata = {
            "total_tasks": total_tasks,
            "large_task_percentage": large_task_percentage,
            "large_tasks": num_large_tasks,
            "small_tasks": num_small_tasks,
            "task_distribution": {
                "read_tasks": sum(1 for task in tasks if task.get('type') == 'read'),
                "write_tasks": sum(1 for task in tasks if task.get('type') == 'write')
            }
        }

        return {
            "metadata": task_set_metadata,
            "categorized_tasks": categorized_tasks,
            "raw_tasks": tasks
        }

    def _generate_tasks(self, num_tasks: int, size: str, start_id: int) -> List[Dict]:
        """Generate tasks of a specific size"""
        tasks = []

        for i in range(num_tasks):
            # Randomly choose between read and write tasks (50-50 distribution)
            task_type = random.choice(["read", "write"])

            # Get appropriate config based on size and type
            config_key = f"{size}_{task_type}_tasks"
            possible_tasks = self.task_configs[config_key]

            # Randomly select a task configuration
            task_name = random.choice(list(possible_tasks.keys()))
            task_config = possible_tasks[task_name]

            # Create task dictionary
            task = {
                "id": start_id + i,
                "task_name": task_name,
                "size": size,
                "type": task_type,
                "instructions": task_config['instr'],
                "data_size": task_config['data'],
                "arrival_time": random.randint(0, 1000),  # Random arrival time
                "status": "pending",
                **{k: v for k, v in task_config.items() if k not in ['instr', 'data']}
            }
            tasks.append(task)

        return tasks

    def _categorize_tasks(self, tasks: List[Dict]) -> Dict[str, List[Dict]]:
        """
        Categorize tasks by their specific characteristics
        """
        categorized = defaultdict(list)

        # Categorize by task names (RT1, RT2, etc.)
        for task in tasks:
            categorized[task['task_name']].append(task)

        return dict(categorized)

def generate_and_save_task_set(total_tasks: int = 1500, large_task_percentage: float = 0.7):
    """
    Generate task set, save to file, and print detailed categorization
    """
    # Create task generator
    generator = TaskGenerator()

    # Generate tasks
    task_set = generator.generate_large_task_set(
        total_tasks=total_tasks,
        large_task_percentage=large_task_percentage
    )

    # Print detailed categorization
    print("\n--- DETAILED TASK CATEGORIZATION ---")
    for task_category, tasks in task_set['categorized_tasks'].items():
        print(f"\n{task_category} Tasks:")
        print(f"Total {task_category} Tasks: {len(tasks)}")
        print("Sample Task Details:")
        for task in tasks[:3]:  # Print first 3 tasks of each category
            print("\nTask Details:")
            for key, value in task.items():
                print(f"{key}: {value}")
        print("-" * 50)

    # Print overall metadata
    print("\n--- TASK SET METADATA ---")
    print(json.dumps(task_set['metadata'], indent=2))

    # Save to JSON
    save_path = 'fcfs_task_set.json'
    with open(save_path, 'w') as f:
        json.dump(task_set, f, indent=2)

    print(f"\nFull task set saved to: {save_path}")

    return task_set

# Run the task generation
if __name__ == "__main__":
    generate_and_save_task_set()



--- DETAILED TASK CATEGORIZATION ---

WT1 Tasks:
Total WT1 Tasks: 270
Sample Task Details:

Task Details:
id: 156
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 401
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 951
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 637
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 123
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 52
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive
--------------------------------------------------

RT2 Tasks:
Total RT2 Tasks: 253
Sample Task Details:

Task Details:
id: 462
task_name: RT2
size: large
type: read
instructions: 4000000
data_size: 0.2
arrival_time: 656
status: pending
cpu_intensity: high
memory_intensity: l

Once we have our stopping criterion, we can finally run our simulation by creating an instance of the `Simulator` class, loading a dataset, and calling the `run_model()` method.

In [7]:

# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

# Checking the placement output
for service in Service.all():
    print(f"{service}. Host: {service.server}")

Service_1. Host: EdgeServer_1
Service_2. Host: EdgeServer_1
Service_3. Host: EdgeServer_1
Service_4. Host: EdgeServer_1
Service_5. Host: EdgeServer_1
Service_6. Host: EdgeServer_1
